## **链接到谷歌云盘**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **1.安装依赖**
完成微调，需要安装openai和环境变量

In [ ]:
!pip install --upgrade openai
!pip install load_dotenv

## **2.引入openai环境**

注意此处需要填写您自己的api_key

In [3]:
import openai
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv('/content/drive/MyDrive/Colab/.env')

openai.api_key  = os.environ['OPENAI_API_KEY']

## **3.数据处理**
将数据格式处理成标准化的JSON-L格式，即Chat completions API中所要求的messages数组形式，每行一个如下所示的JSON对象。
``` JSON
{"messages": [{"role": "system", "content": ""}, {"role": "user", "content": ""}, {"role": "assistant", "content": ""}]}
```

In [7]:
import json
file_raw_path = '/content/drive/MyDrive/openai/data/raw/training_data.txt'
file_processed_path = '/content/drive/MyDrive/openai/data/processed/chat_data.jsonl'
# 打开收集的数据文件
with open(file_raw_path, 'r', encoding='utf-8') as file:
    content = file.read()

# 以三个连续的换行符分割成段落
paragraphs_level1 = content.split('\n\n\n')

# 构建消息列表
messages = []

for i, paragraph_level1 in enumerate(paragraphs_level1, start=1):
    # 在每个段落内再以两个连续的换行符分割内容
    paragraphs_level2 = paragraph_level1.split('\n\n')

    # 初始化消息对象
    message = {"messages": []}

    # 添加系统消息
    message["messages"].append({"role": "system", "content": "你是一名翻译"})

    # 添加第二级段落内容到不同角色的消息
    for j, paragraph_level2 in enumerate(paragraphs_level2, start=1):
        role = "user" if j % 2 == 1 else "assistant"
        message["messages"].append({"role": role, "content": paragraph_level2})

    # 将消息对象添加到消息列表
    messages.append(message)

# 将消息列表写入到 chat_data.jsonl 文件
with open(file_processed_path, 'w', encoding='utf-8') as jsonl_file:
    for message in messages:
        jsonl_file.write(json.dumps(message, ensure_ascii=False) + '\n')


## **4.上传数据**
将上一步处理完成的JSON-L格式数据，通过files接口上传到OpenAI服务器，获取文件id

In [ ]:
from openai import OpenAI
client = OpenAI()
train_file_info = client.files.create(
    file=open(file_processed_path, "rb"),
    purpose='fine-tune'
)
print(train_file_info.status)

## **5.创建fine-tuning任务**
通过fine_tuning/jobs接口，将上一步上传的数据文件id传入，创建fine-tuning任务，并获取任务id

In [ ]:
file_tuning_job = client.fine_tuning.jobs.create(
    training_file=train_file_info.id,
    model="gpt-3.5-turbo"
)
print(file_tuning_job.status)

## **6.查看微调进度**
通过fine_tuning/jobs接口，传入上一步获取的微调任务id，获得进行中的微调任务状态。
如果status的值时running，说明该模型还在训练中，此时fine_tuned_model里面没有值，需要继续等待。

如果status是succeeded，说明已经训练完成，此时通过fine_tuned_model的值，可以得到微调成功的模型id

如果训练成功，开发者账号对应的邮箱，也会收到一封电子邮件，里面会包含模型id。

In [ ]:
file_info = client.fine_tuning.jobs.retrieve(file_tuning_job.id)
print(file_info.status)

## **7.测试模型**
通过Completion API，对微调成功的模型进行效果测试。
我们可以按正常GPT模型，例如GPT-3.5-turbo的调用方式，来调用微调好的模型，访问参数完全相同。

In [ ]:
completion = client.chat.completions.create(
    model=file_info.fine_tuned_model,
    messages=[
        {"role": "system", "content": "你是一名翻译 "},
        {"role": "user", "content": "全球最大的中文搜索引擎、致力于让网民更便捷地获取信息,找到所求。百度超过千亿的中文网页数据库,可以瞬间找到相关的搜索结果"}
    ],
    temperature=0
)
print(completion.choices[0].message.content)

## 总结
至此，一个免安装的微调流程就完成了。需要注意的是，其中用到的数据样本较少，仅用于演示微调的技术实现过程。当然，即使很少的数据也能完成微调，这是GPT-3.5-turbo的优势，但从工作的角度来看，我们仍有必要认真的准备和标记数据。